# Zakupki website scraping for Piotr

The aim of this notebook is to scrape details of each contract hosted on the Russian Zakupki public sector contract awarding website.

The input for this project will be the Zakupki URL. This code can be run on different dates to pull fresh contract data.

Method:
1.   Identify the number of pages of contracts to be scraped (using the contract filters provided).
2.   Iterate through each page, scraping the registration number of each contract.
3.   Access the website for each contract by placing the registraion number in the URL.
4.   Scrape the details for each contract and add them to a list of Contracts dataclasses.
5.   Format these Contract objects as a dataframe and output the dataframe to a csv file.


The output of this project will be the CSV file, with each row representing a new contract from the webstie.


### Section 1: Setup

In [2]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from datetime import date
from dataclasses import dataclass
from tqdm import tqdm
from dateutil import parser
from threading import Thread
import pandas as pd
import datetime
from datetime import date, timedelta
import logging
import http.client
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor
import math
from os import walk
import json


In [3]:
logging = False

if logging:

    http.client.HTTPConnection.debuglevel = 1

    # You must initialize logging, otherwise you'll not see debug output.
    logging.basicConfig()
    logging.getLogger().setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

### Section 2: Determine Number of pages to scrape
Test connection to the website and determine number of pages to scrape

In [4]:
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

@lru_cache(maxsize=None)
def getPage(tempURL):
  # If User-Agent is not set to custom, the website will know a Python script is accessing it and block some of the request

  response = session.get(tempURL, headers={'User-Agent': 'Custom'})
  return BeautifulSoup(response.content, "html.parser")

In [34]:
# Getting the dates we want to scrape.

url="https://zakupki.gov.ru/epz/contract/search/results.html?morphology=on&search-filter=%D0%94%D0%B0%D1%82%D0%B5+%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D1%8F&fz44=on&contractStageList_0=on&contractStageList_1=on&contractStageList=0%2C1&contractPriceFrom=1000000&contractCurrencyID=-1&budgetLevelsIdNameHidden=%7B%7D&customerPlace=5277383&customerPlaceCodes=66000000000&contractDateFrom={}&contractDateTo={}&selectedLaws=FZ44&sortBy=UPDATE_DATE&pageNumber={}&sortDirection=false&recordsPerPage=_500&showLotsInfoHidden=false"
start = date(2016, 1, 1)
end = date(2022, 6, 12)
days = timedelta(days=1)

calendar=[]

while start<=end:
  calendar.append(start.strftime('%d.%m.%Y'))
  start+=days

print("Created {} dates".format(len(calendar)))

Created 2355 dates



### Section 3: Scrape each registration number

Scrape the reg numbers of each contract, so they can be accessed individually

In [6]:
class Page:

    def __init__(self, day, pageNum, pagefile):

        self.day = day
        self.pageNum = pageNum
        self.pagefile = pagefile


In [7]:
def getContracts(page):

    regNumbersList = []

    # Obtain a list of all the sections of HTML containing a contract in the web page
    listOfContracts = page.find_all("div", {"class": "registry-entry__header-mid__number"})
    
    # Segment the registration number from the URL of each contract
    for contract in listOfContracts:
        href = contract.find("a")['href']
        regNum = href[href.index("Number=")+7:]
        regNumbersList.append(regNum)

    return regNumbersList

In [8]:
def progress(idx, data):

    x_ = int(((idx+1) * 100) / len(data))
    y_ = idx % math.ceil(len(data) / 10)
    
    print(" ----\n{}% completed\n----".format(x_)) if y_ == 0 else None

In [30]:
# Getting the web page for all the contracts for each date in the range we want to scrape.

regNumbersDict = {}

def getRegNumbersForDate(i, day):

  if day in regNumbersDict:
    return

  tempURL = url.format(day, day, 1)

  page = getPage(tempURL)

  # Scrape the max number of pages
  try:
    maxPageNum = int(page.select('a[data-pagenumber]')[-2].find("span").text)
    print("{} pages for this day".format(maxPageNum))
  except:
    maxPageNum = 1


  # Leave my variable names alone :(
  totalRegNumbersForThisDay = 0

  for i in range(1, maxPageNum+1):

    # Creating a temporary URL for each page containing contracts
    tempPageURL = url.format(day, day, i)

    # Request the page and format it as a BeautifulSoup object so that we can perform scrapings
    page = getPage(tempPageURL)

    regNumbersList = getContracts(page)

    totalRegNumbersForThisDay += len(regNumbersList)

    regNumbersDict[day] = regNumbersList


  print("Fetched day {} had {} contracts \n".format(day, totalRegNumbersForThisDay), end='')
  
  # progress(i, calendar)


In [35]:

## This now has regNumbers caching too 

cachedRegNums = {}

# load the data from the json file
with open('cachedRegNums.json', 'r') as f:
  cachedRegNums = json.load(f)


with ThreadPoolExecutor(max_workers=100) as ex:
  for i, day in enumerate(calendar):
    if day in cachedRegNums:
      regNumbersDict[day] = cachedRegNums[day]
      print("Cached day {} had {} contracts \n".format(day, len(regNumbersDict[day])), end='')
    else:
      ex.submit(getRegNumbersForDate, i, day)

    
combinedRegNumbersDict = cachedRegNums | regNumbersDict

# print(combinedRegNumbersDict)


with open('cachedRegNums.json', 'w') as f:
  json.dump(combinedRegNumbersDict, f)


tempRegNumbers = list(regNumbersDict.values())

regNumbers = []

for t in tempRegNumbers:
  regNumbers.extend(t)

# print(regNumbers)

print("------------------- \n {} contracts found in total".format(len(regNumbers)))

Cached day 01.01.2016 had 80 contracts 
Cached day 02.01.2016 had 4 contracts 
Cached day 03.01.2016 had 2 contracts 
Cached day 04.01.2016 had 4 contracts 
Cached day 05.01.2016 had 1 contracts 
Cached day 06.01.2016 had 5 contracts 
Cached day 07.01.2016 had 0 contracts 
Cached day 08.01.2016 had 3 contracts 
Cached day 09.01.2016 had 1 contracts 
Cached day 10.01.2016 had 2 contracts 
Cached day 11.01.2016 had 115 contracts 
Cached day 12.01.2016 had 41 contracts 
Cached day 13.01.2016 had 23 contracts 
Cached day 14.01.2016 had 25 contracts 
Cached day 15.01.2016 had 25 contracts 
Cached day 16.01.2016 had 1 contracts 
Fetched day 17.01.2016 had 0 contracts 
Fetched day 19.01.2016 had 34 contracts 
Fetched day 20.01.2016 had 19 contracts 
Fetched day 24.01.2016 had 1 contracts 
Fetched day 21.01.2016 had 21 contracts 
Fetched day 22.01.2016 had 20 contracts 
Fetched day 30.01.2016 had 2 contracts 
Fetched day 23.01.2016 had 3 contracts 
Fetched day 31.01.2016 had 7 contracts 
Fetch

The below is just for testing labels etc.

In [ ]:
# url="https://zakupki.gov.ru/epz/contract/contractCard/common-info.html?reestrNumber=3662310492216000051"
# soup=getPage(url)
# secondtab=soup.findAll("span", {"class":"grey-main-light"})[-2]
# print(secondtab)
# # # # products=soup.findAll("span",{"class":"section__info"})
# # # product=products[24].text.strip()
# # print(product)

# # # # # ### first tab info ###
# # table=soup.findAll("td",{"class":"tableBlock__col"})
# # # # cardinfo=soup.findAll("span",{"class":"cardMainInfo__content"})
# # # signed=cardinfo[3].text.strip()
# # # deadline=cardinfo[4].text.strip()
# # # # price=soup.find("span", {"class":"cardMainInfo__content cost"}).text.replace("\xa0","").replace(",",".").replace("₽","").strip()
# # print(signed)
# # # org=table[0].text.strip().split("\n")
# # # procurer=org[0]
# # # INN=org[9]
# # # KPP=org[13]
# # # registered=org[17]

# try:
#     address=table[3].text.strip()
#     numbermail=table[3].text.strip().split("\n")
# #     number=numbermail[0]
# #     mail=numbermail[1].strip()

# # except:
# #     address=table[2].text.strip()
# #     numbermail=table[2].text.strip().split("\n")
# #     number=numbermail[0]
# #     mail=numbermail[1].strip()

# table=soup.findAll("section",{"class":"blockInfo__section section"})
# # # check=table[6]
# # # print(check)

# # section=soup.findAll("span",{"class":"section__info"})
# # # method=section[3].text.strip()
# # # print(method)
# # # # print(method)
# # # # # # ### second tab info ###
# # print((table))

# try:
#   check=table[3]
#   method=check.findAll('span', {'class':'section__info'})[0].text.strip()

# except:
#   method=section[6].text.strip()

# print(method)

# # # secondtab=soup.findAll("div", {"class": "col"})
# # # # print(secondtab)
# # # tables = []
# for tab in secondtab:
#     tables.append(tab.findAll("tbody"))

# object=tables[-1][0].findAll("td",{"class":"tableBlock__col"})
# code=object[1].text.strip().split("\n")[-1].strip()
# print(code)


### Section 4: Details scraping

The Contract Dataclass will store the information during scraping.
If any information can't be scraped, default values have been provided in their place

In [ ]:
@dataclass
class Contract:

  # TODO: Add reg number to class

  # Main Section
  id: float = 0
  price: float = 0.0
  signed: date = None
  deadline: date = None

  # Tab 1
  method: str = "none"
  procurer: str = "none"
  supplier: str = "none"
  proinn: str = "none"
  supinn: str = "none"
  registered: date = None
  address: str = "none"
  number: str = "none"
  mail: str = "none"

  # Tab 2
  code: float = 0.0
  product: str = "none"
  

  def __repr__(self):
    return "\nContract id= {} \n First tab: price={}, signed={}, deadline={}, method={}, procurer={}, supplier={}, proinn={}, supinn={}, registered={}, address={}, number={}, mail={} \n  Second tab: code={}, product={})".format(self.id, self.price, self.signed, self.deadline, self.method, self.procurer, self.supplier, self.proinn, self.supinn, self.registered, self.address, self.number, self.mail, self.code,  self.product)

  

Method for scraping the data from each contract

In [ ]:
def getSectionDict(page):

    sections=page.findAll("section",{"class":"blockInfo__section section"})

    # Turning the sections into a dictionary that will be easier to work with.
    titles = [key.findAll("span")[0] for key in sections]
    values = [key.findAll("span")[1] for key in sections]

    sectionDict = {titles[i].text.strip() : values[i].text.strip() for i in range(len(titles))}

    return sectionDict


def getTableDict(page, secondTab=False):


    if secondTab:
        sectionOfInterest = page.findAll("div", {"class": "col"})[-1]
    else:
        sectionOfInterest = page

    table = sectionOfInterest.findAll("tr",{"class":"tableBlock__row"})

    headers = [i.text.strip() for i in table[0].findAll("th", {"class":"tableBlock__col tableBlock__col_header"})]
    data = [list(filter(None, [j.strip() for j in i.text.split("\n")])) for i in table[1].findAll("td")]
  
    tableDict = {headers[i] : data[i] for i in range(len(headers))}

    return tableDict

In [ ]:
def scrapeData(reg):

  # try:

    # Input: reg = one registration number.

    # Different URL from the one above, this accesses more information from Zakupki.
    dir = "https://zakupki.gov.ru/epz/contract/contractCard/{}.html?reestrNumber={}"

    # Getting the web page for the given contract
    tempDir = dir.format("common-info", reg)
    page = getPage(tempDir)

    # We probably don't need this with the method I've used below.
    contractTypeTwo = False

    # Enter the text here that should be present to signify the second type of contract.
    if page.findAll(text="Основание заключения контракта с единственным поставщиком"):
      contractTypeTwo = True
      
    id = reg  
    sectionDict = getSectionDict(page)
    firstTableDict = getTableDict(page)

    # print(sectionDict)
    # print(firstTableDict)

    # print(sectionDict, firstTableDict)
    # try:
    #   price=sectionDict["Цена контракта"].replace("\xa0","").replace(",",".").replace("₽","").strip().split()[0]
    # except:
    #   price=sectionDict["Ориентировочное значение цены контракта"].replace("\xa0","").replace(",",".").replace("₽","").strip().split()[0]
    #   try:
    #     price=sectionDict["Максимальное значение цены контракта"].replace("\xa0","").replace(",",".").replace("₽","").strip().split()[0]
    #   except:
    price=page.find('span', {'class':'cardMainInfo__content cost'}).text.strip().replace("₽","").replace(" ","")
      
    signed=sectionDict["Дата заключения контракта"].split()[0]
    deadline=sectionDict["Дата окончания исполнения контракта"].split()[0]
    
    ### fixed issue with method ### 
    try:
      method = sectionDict["Способ определения поставщика (подрядчика, исполнителя)"]
    except:
      if page.findAll(text="Основание заключения контракта с единственным поставщиком"):
            method="Закупка у единственного поставщика (подрядчика, исполнителя)"
            
    procurer=sectionDict["Полное наименование заказчика"]
    supplier=firstTableDict["Организация"][0]

    proinn=sectionDict["ИНН"]

    ### fixed issues for missing values sometimes in the table ###
    
    registered=firstTableDict["Организация"][-1]
    
    ### fixed, testing ###   
    try: 
      if firstTableDict["Организация"][-4]=="КПП:":
          supinn=firstTableDict["Организация"][-5]
      else:
          supinn=firstTableDict["Организация"][-3]
    except:
      supinn=""

    ### fixed issues in lower table ### 
    
    try:
      address=firstTableDict["Адрес места нахождения"]
    except:
      address=firstTableDict["Адрес в стране регистрации"]
    number=firstTableDict["Телефон, электронная почта"][0]
    mail=firstTableDict["Телефон, электронная почта"][1]

    ### details about winner - ALSO, THERE'S OPTION TO SCRAPE SUBCONTRACTORS ### 

    ### Second tab ###
    tempDir = dir.format("payment-info-and-target-of-order", reg)
    page = getPage(tempDir)

    ### code stands for the product code, which can be later identified to return industry type ### 
    
    secondTableDict = getTableDict(page, True)
    
    try:
      code = secondTableDict["Позиции по КТРУ, ОКПД2"][1][1:-1]
    except:
      code = ''
        
    product = secondTableDict["Наименование объекта закупки и его характеристики"][0]
    
    # Create the Contract dataclass object and append it to a list of objects.
    # This method means that missing data can be accounted for.
    # print(method)

    contract = Contract(id=id, price=price, signed=signed, deadline=deadline, method=method, procurer=procurer, supplier=supplier, proinn=proinn, supinn=supinn, registered=registered, address=address, number=number, mail=mail, code=code, product=product)
    
    # contracts.append(contract)
    print('Completed {}'.format(id))
    return contract
  # except Exception as e:
  #   failedRegNumbers.append(reg)
  #   print("Failed to scrape {}".format(reg))
  #   print(e)

### Section 5: Starting execution
Scrape the contracts themselves using threading

In [ ]:
def scrape(reg, i):
    
    try:
        _ = int(reg)
        # print("Scraping {}".format(reg))
        scrapeData(reg)
    except TypeError:
        
        # TODO make 500 contracts change here.
        
        contracts = []

        for idx, r in enumerate(reg):
            # print("Scraping {}".format(r))
            contracts.append(scrapeData(r))
            # progress(i + idx, regNumbers)

        tempDF = pd.DataFrame(contracts)
        tempDF.to_pickle('./data/contracts_{}.pkl'.format(i))

In [ ]:
progressNum = 0

failedRegNumbers = []

# regNumbers = [3666404326716000002, 3666404326716000002, 1722300731616000034, 1722300731616000034, 2666001041516000469]

print("Starting scrape with {} reg numbers\n".format(len(regNumbers)))

# scrape(regNumbers[:10])

# for contract in contracts:
#     print(contract)

# for regNumber in tqdm(regNumbers[:50]):
#   thread = Thread(target = scrapeData, args = (regNumber,))
#   thread.start()

interval = 15

# scrape(regNumbers)

# TODO Make it do eg 500 at a time, save to a text file.

with ThreadPoolExecutor(max_workers=10) as ex:
    for i in range(0, len(regNumbers), interval):
        tempNumbers = regNumbers[i:i+interval]
        # print("Scraping ", tempNumbers)
        ex.submit(scrape, tempNumbers, i)

# print("Scraped {} contracts".format(len(contracts)))
print("Failed to scrape {} contracts".format(len(failedRegNumbers)))
print(failedRegNumbers) if len(failedRegNumbers) > 0 else None



Starting scrape with 6928 reg numbers

Completed 3663201757816000005
Completed 3662400704716000008
Completed 3664900138216000003
Completed 1668503165716000008
Completed 1665808957716000012
Completed 3662400704716000002
Completed 2665831461716000017
Completed 3665100278516000004
Completed 3660900842316000003Completed 3665400825616000001

Completed 3662600951516000001
Completed 3663100638016000001
Completed 2662309705516000036
Completed 3666801180516000003
Completed 3661700951916000005
Completed 3665401323116000001
Completed 2665831461716000018
Completed 3667409225616000002
Completed 2661900214916000015
Completed 2666801189016000001
Completed 1666015542816000004
Completed 3661500671316000003
Completed 3662501747116000003
Completed 1662300085016000001
Completed 3661202214616000002
Completed 2665800623516000012
Completed 1667007300516000012
Completed 2660200751916000035
Completed 2666001041516000037
Completed 2667600280416000020
Completed 3664600920016000003
Completed 2666100219916000064
C

KeyboardInterrupt: 

Completed 2666208106616000022
Completed 1666105699816000039
Completed 3662310208016000131
Completed 1665800007116000128
Completed 1665800007116000149
Completed 3665905173816000002
Completed 3662501728816000017
Completed 3660601939216000012
Completed 2665809196016000021Completed 2666208106616000023

Completed 3662310208016000130
Completed 2662309705516000188
Completed 3662600356016000015
Completed 1665800007116000135
Completed 2666001041516000167
Completed 3667131029816000019
Completed 2665846316816001017
Completed 1666101464516000118
Completed 2665801491016000042
Completed 2666208106616000027
Completed 2666001041516000179
Completed 1662801020216000006
Completed 3662310208016000128
Completed 1665800007116000129
Completed 1666402342216000017


### Now that the data is saved to the hard disk, we can run the below code without needing to rerun the scraping process

In [ ]:
from os import walk
import pandas as pd

contracts = []

for (dirpath, dirnames, filenames) in walk('./data'):
    for filename in filenames:
        if filename.endswith('.pkl'):
            tempDF = pd.read_pickle('./data/' + filename)
            # print(tempDF)
            contracts.append(tempDF)
            
finalDF = pd.concat(contracts, ignore_index=True)

print(finalDF.head())

                    id           price      signed    deadline  \
0  4661201154616000016  12 325 742,00   29.02.2016  19.08.2016   
1  1666001327916000095   2 500 227,25   29.02.2016  31.12.2016   
2  2665841709816000004  87 083 334,44   29.02.2016  30.11.2016   
3  1666101461316000004   1 485 728,16   29.02.2016  31.12.2016   
4  4661201154616000004  12 325 742,00   29.02.2016  30.06.2016   

                method                                           procurer  \
0  Электронный аукцион  ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО "УРАЛЬСКОЕ ПРОЕК...   
1  Электронный аукцион  ГОСУДАРСТВЕННОЕ УЧРЕЖДЕНИЕ - СВЕРДЛОВСКОЕ РЕГИ...   
2  Электронный аукцион  УПРАВЛЕНИЕ ДЕЛАМИ ГУБЕРНАТОРА СВЕРДЛОВСКОЙ ОБЛ...   
3  Электронный аукцион  УПРАВЛЕНИЕ МИНИСТЕРСТВА ВНУТРЕННИХ ДЕЛ РОССИЙС...   
4  Электронный аукцион  ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО "УРАЛЬСКОЕ ПРОЕК...   

                                            supplier      proinn      supinn  \
0  ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО "КОМПАНИЯ "НТНК"...  661201

### Section 6: Output

Convert the list of contract classes to a dataframe so that they can be exported to a csv file

In [ ]:
finalDF.to_csv("zakupki.csv")